<h4>本节深入探讨结构化API</h4>

- 结构化API可以处理非结构化的日志文件、半结构化的CSV文件，以及高度结构化的Parquet文件
- 结构化API通常指以下三种核心分布式集合类型的API
    - Dataset类型
    - DataFrame类型
    - SQL表和视图
    


<h4>DataFrame类型和DataSet类型</h4>

- 具有行和列的类似于（分布式）数据表的集合类型
- 可以通过它指定对特定位置数据的操作，该操作以惰性评估方式执行
- 表和视图于DataFrame基本相同，所以通常在DataFrame上执行SQL操作，而不是用DataFrame专用执行代码

<h4>Schema</h4>

- 定义了DataFrame的列名和类型

<h4>结构化Spark类型概述</h4>

- Spark有自己的编程语言，内部使用名为Catalyst的引擎
- Spark在计划制定和执行作业的过程中使用Catalyst维护自己的类型信息，这样能够带来优化空间，提高性能
- 我们可以使用其他语言，是由于Spark将其类型直接映射到不用语言的API
- 因此我们大多数情况下都在操作Spark类型而非Python类型
- 以下代码是在Spark中执行加法而非Python或Scala

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python").getOrCreate()

In [16]:
df = spark.range(500).toDF("number")
df.select(df["number"]+10).limit(1).show()

+-------------+
|(number + 10)|
+-------------+
|           10|
+-------------+



<b>需要特别指出的是DataFrame的每条记录，即每一行都是Row类型对象</b>

In [17]:
df.select(df["number"]+10).limit(1).collect()

[Row((number + 10)=10)]

<h4>DataFrame与Dataset的比较</h4>

- 结构化API的两类，非类型化的DataFrame和类型化的Dataset
- DataFrame是有类型的，只是它的类型是完全交给Spark维护的
- 这意味着只有在运行时，才会检查DataFrame的类型和schema中指定的类型是否一致
- 与之相对的，DataSet在编译时就会检查类型是否符合规范
- DataSet仅适用于基于Java虚拟机（JVM）的语言（比如Scala和Java）

<h4>Spark类型映射Python类型参考表</h4>

|数据类型|Python的值类型|获取或创建数据类型的API
|:----|:----|:----|
|ByteType|int或long，数字在运行时转换为1字节带符号整数，确保数字在-128~127的范围内|ByteType()
|ShortType|2字节，-32768~32767|ShortType()
|IntegerType|定义宽松，数字太大最好使用LongType()|IntegerType()
|LongTtype|长得很|LongType()
|FloatType|数字将被转换为4字节的单精度浮点数|FloatType()
|DoubleType
|DecimalType
|StringType
|BinaryType
|BooleanType
|TimestampType
|DataType
|ArrayType
|MapType|字典|MapType(keyType,valueType,[valueContainsNull])，valueContainsNull默认为True
|StrucType|列表或元组|StructType(fields)。fields是包含多个StructField的list
|StructField|该字段对应的Python数据类型|StructField(name,dataType,[nullable])

<h4>结构化API执行概述——用户代码到执行代码过程步骤</h4>

1. 编写DataFrame/DataSet/SQL代码
2. 如果代码能有效执行，Spark将其转换为一个逻辑执行计划（Logical Plan）
3. Spark检查可行的优化策略并优化，优化交给Catalyst优化器，将此逻辑执行计划转化为一个物理执行计划（Physical Plan）
4. Spark在集群上执行该物理执行计划（RDD操作）

<h4>逻辑计划</h4>

- 仅代表一组抽象转换，不涉及执行器或驱动器，